In [ ]:
# let us import all the necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# import the beautifulsoup to scrape the web page
from bs4 import BeautifulSoup # to scrape the data
# import the request 
import requests # library to handle requests

print('All the libraries are now imported.')

All the libraries are now imported.


In [ ]:
# let us download the New York City dataset in .json format from the sources
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('The New York City data downloaded!')

The New York City data downloaded!


In [ ]:
# let us load and view the newyork_data.json
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

In [ ]:
# let us get the relevant data stored in the 'features' key of the json and view the first one
neighbourhoods_data = newyork_data['features']
neighbourhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [ ]:
# let us define the dataframe columns
column_names = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 

# let us instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)

# let view it
neighbourhoods

,Borough,Neighbourhood,Latitude,Longitude


In [ ]:
# let us loop through the data to fill the dataframe above
for data in neighbourhoods_data:
    # get the borough
    borough = data['properties']['borough']
    # get neighbourhood names
    neighbourhood_name = data['properties']['name']
    # get the latitude and longitude coordinate   
    neighbourhood_latlon = data['geometry']['coordinates']
    # get the latitude
    neighbourhood_lat = neighbourhood_latlon[1]
    # get the longitude
    neighbourhood_lon = neighbourhood_latlon[0]
    # append it to the dataframe created earlier, the neighbourhoods
    neighbourhoods = neighbourhoods.append({'Borough': borough,
                                          'Neighbourhood': neighbourhood_name,
                                          'Latitude': neighbourhood_lat,
                                          'Longitude': neighbourhood_lon}, ignore_index=True)

In [ ]:
# let us view the first five
neighbourhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [ ]:
# let us see the number of borough and neighbourhoods
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(neighbourhoods['Borough'].unique()),
        neighbourhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighbourhoods.


In [ ]:
# let us get manhattan data
manhattan_data = neighbourhoods[neighbourhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [ ]:
# let us download the Toronto City dataset from the sources
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# get the request and get the content
soup = BeautifulSoup(res.content,'lxml')
# find the table in the content
table = soup.find_all('table')[0] 
# turn the table to dataframe
table_df = pd.read_html(str(table))
# get the first table 
df = table_df[0]
#Renaming the Column 'Postal Code'
df.rename(columns={'Postal Code':'Postcode'},inplace=True)
df = df.groupby('Postcode').agg({'Borough': 'first', 'Neighbourhood':','.join }).reset_index()
dframe=df[(df['Borough']!='Not assigned') & (df['Neighbourhood']!='Not assigned')]
dframe
dframe.head()

,Postcode,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [ ]:
#Get the postcode and convert the Postcode to list
lats=[]
lngs =[]
lats.clear()
lngs.clear()
postcodes = dframe['Postcode'].tolist()
gsdata_df=pd.read_csv('https://cocl.us/Geospatial_data')
postalcodes=gsdata_df['Postal Code'].tolist()
latitudes=gsdata_df['Latitude'].tolist()
longitudes=gsdata_df['Longitude'].tolist()
zp=zip(postalcodes,latitudes,longitudes)
for ps,la,lg in zp:
  for postcode in postcodes:
    if postcode == ps:
      lats.append(la)
      lngs.append(lg)
print(len(lats))
print(len(lngs))

103
103


In [ ]:
dframe['Latitude']= lats
dframe['Longitude']=lngs
dframe.head()    #append({'Borough': borough,
                                          #'Neighborhood': neighborhood_name,
                                          ##'Latitude': neighborhood_lat,
                                          #'Longitude': neighborhood_lon}, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postcode,Borough,Neighbourhood,Latitude,Longitude
1,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
4,M1G,Scarborough,Woburn,43.770992,-79.216917
5,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [ ]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(dframe['Latitude'], dframe['Longitude'], dframe['Borough'], dframe['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [ ]:
borough_with_toronto=dframe[dframe.loc[:,'Borough'].str.contains('Toronto')].reset_index(drop=True)
borough_with_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [ ]:
borough_with_toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(borough_with_toronto['Latitude'], borough_with_toronto['Longitude'], borough_with_toronto['Borough'], borough_with_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(borough_with_toronto_map)  
    
borough_with_toronto_map

In [ ]:
CLIENT_ID = 'CCPOPTCWRXNJ4DOYW254JTAZSWQR4MOUMQ1WKIYGK2EOF50D'

CLIENT_SECRET='Q1MKRAP5DVAZZ3WWOOAIESPEZ5IMAKH4EDID5M1JNHKLBRW5' # your Foursquare ID # your Foursquare Secret
VERSION = '20201018' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CCPOPTCWRXNJ4DOYW254JTAZSWQR4MOUMQ1WKIYGK2EOF50D
CLIENT_SECRET:Q1MKRAP5DVAZZ3WWOOAIESPEZ5IMAKH4EDID5M1JNHKLBRW5


Explore the First Neighbourhood in the DataFrame

In [ ]:
borough_with_toronto.loc[0, 'Neighbourhood']

'The Beaches'

In [ ]:
neighborhood_latitude = borough_with_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = borough_with_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = borough_with_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


# Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [ ]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CCPOPTCWRXNJ4DOYW254JTAZSWQR4MOUMQ1WKIYGK2EOF50D&client_secret=Q1MKRAP5DVAZZ3WWOOAIESPEZ5IMAKH4EDID5M1JNHKLBRW5&v=20201018&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [ ]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f8da5f90d71e67f1c44e07e'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 89,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


Create a function to repeat the same process to all the neighborhoods in with word 'Toronto'


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
   for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([i  ])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', X  
                   'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) 
    venues_list=[]

IndentationError: ignored